# Métrica 1.1.07 - CP vs Domicilio Empleo

In [10]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
# Asumimos que tienes un archivo config.py con tus variables de conexión
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_07_CP_EMPLEO_VS_DOMICILIO" 
CATALOG_COLLECTION_NAME = "catalogo_cp"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

# Conjunto de "palabras vacías" que ignoraremos en la comparación de colonias.
# Estas son palabras comunes que no aportan identidad única.
STOP_WORDS_COLONIAS = {
    "COLONIA", "FRACCIONAMIENTO", "UNIDAD", "HABITACIONAL", "BARRIO", "ZONA",
    "RESIDENCIAL", "PUEBLO", "RANCHO", "RANCHERIA", "GRANJA", "EJIDO",
    "DE", "LA", "LAS", "LOS", "EL", "Y", "EN",
    "1RA", "2DA", "3RA", "4TA", "5TA", "SECCION",
    "AMPLIACION", "PRIVADA", "ANDADOR", "CALLE", "AVENIDA",
    "NORTE", "SUR", "ESTE", "OESTE", "PONIENTE", "ORIENTE"
}

def normalize_string(s: str) -> str:
    """
    Quita acentos, caracteres especiales y convierte a mayúsculas para comparar.
    """
    if not isinstance(s, str): return ""
    
    # 1. Convertir a mayúsculas PRIMERO
    s = s.upper()
    
    # 2. Quitar acentos (ej. CARIÑÁN -> CARINAN)
    s = ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn')
                
    # 3. Quitar caracteres no alfanuméricos (deja letras, números y espacios)
    s_norm = re.sub(r'[^A-Z0-9\s]', '', s)
    
    # 4. Quitar espacios dobles y de los extremos
    return " ".join(s_norm.split())

def validar_cp_en_mongo(db, cp: str, colonia_decl: str, municipio_decl: str) -> str:
    """
    Valida la congruencia del CP, Colonia y Municipio contra
    nuestra colección 'catalogo_cp' en MongoDB.
    ¡CON LÓGICA DE COINCIDENCIA DE PALABRAS CLAVE!
    """
    
    if not all([cp, colonia_decl, municipio_decl]):
        return "SIN_DATO"

    cp_norm = str(cp).strip().zfill(5)
    colonia_norm = normalize_string(colonia_decl)
    municipio_norm = normalize_string(municipio_decl)

    if not all([cp_norm, colonia_norm, municipio_norm]):
        return "SIN_DATO"

    catalogo_collection = db[CATALOG_COLLECTION_NAME] 
    
    registros_validos = list(catalogo_collection.find({"d_codigo": cp_norm}))

    if not registros_validos:
        return "NO_CUMPLE_CP" 

    # --- ¡NUEVA LÓGICA DE PALABRAS CLAVE! ---
    
    # 1. Convertir la colonia declarada en un set de palabras clave
    palabras_declaradas = set(colonia_norm.split())
    # 2. Quitar las palabras vacías
    set_declarado = palabras_declaradas - STOP_WORDS_COLONIAS

    # Si después de quitar las palabras vacías no queda nada,
    # (ej. declaró "Colonia Centro"), usamos el set original.
    if not set_declarado:
        set_declarado = palabras_declaradas

    encontrado = False
    for item in registros_validos:
        colonia_oficial = normalize_string(item.get("d_asenta"))
        municipio_oficial = normalize_string(item.get("D_mnpio"))
        
        # 3. Comparamos municipios (debe ser exacto)
        if municipio_norm == municipio_oficial:
            
            # 4. Convertir la colonia oficial en un set de palabras clave
            palabras_oficiales = set(colonia_oficial.split())
            # 5. Quitar las palabras vacías
            set_oficial = palabras_oficiales - STOP_WORDS_COLONIAS
            
            if not set_oficial:
                set_oficial = palabras_oficiales

            # 6. Buscar la intersección (palabras en común)
            interseccion = set_declarado.intersection(set_oficial)
            
            # 7. Si hay al menos una palabra en común, es CUMPLE
            if len(interseccion) > 0:
                encontrado = True
                break
            
    return "CUMPLE" if encontrado else "NO_CUMPLE"

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_cp_empleo():
    """
    Procesa TODAS las declaraciones para la Métrica 1.1.07 (Domicilio Empleo)
    """
    client = None
    total_documentos, cumple, no_cumple_cp, no_cumple, sin_dato = 0, 0, 0, 0, 0
    
    try:
        print(f"\nConectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        if CATALOG_COLLECTION_NAME not in db.list_collection_names():
            print(f"--- ERROR CRÍTICO ---")
            print(f"La colección '{CATALOG_COLLECTION_NAME}' no existe.")
            print(f"Por favor, ejecuta primero el script para poblar el catálogo de CP.")
            print("---------------------")
            return

        print("¡Conexión exitosa!")
        print(f"--- INICIANDO PROCESAMIENTO (Métrica 1.1.07 - CP vs Domicilio Empleo) ---")

        proyeccion = {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico": 1
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]
            cp, colonia, municipio = None, None, None
            
            try:
                domicilio = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosEmpleoCargoComision", {}).get("domicilioMexico")
                if isinstance(domicilio, dict):
                    cp = domicilio.get("codigoPostal")
                    colonia = domicilio.get("coloniaLocalidad")
                    municipio_obj = domicilio.get("municipioAlcaldia")
                    if isinstance(municipio_obj, dict):
                        municipio = municipio_obj.get("valor")
            except AttributeError:
                pass 

            resultado_metrica = validar_cp_en_mongo(db, cp, colonia, municipio)
            
            if resultado_metrica == "CUMPLE": cumple += 1
            elif resultado_metrica == "NO_CUMPLE": no_cumple += 1
            elif resultado_metrica == "NO_CUMPLE_CP": no_cumple_cp += 1
            else: sin_dato += 1

            print(f"\nProcesando Documento ID: {original_id}")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > [Empleo] CP: '{cp}', Colonia: '{colonia}', Municipio: '{municipio}'")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")

            filtro = { "_id": original_id }
            actualizacion = { "$set": { METRIC_ID: resultado_metrica } }
            target_collection.update_one(filtro, actualizacion, upsert=True)
            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.07):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE' (Col/Mun no coinciden): {no_cumple}")
        print(f"  > Métrica 'NO_CUMPLE_CP' (CP no existe): {no_cumple_cp}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e: print(f"Ocurrió un error inesperado: {e}")
    finally:
        if client: client.close(); print("Conexión cerrada.")

if __name__ == "__main__":
    procesar_todas_las_declaraciones_cp_empleo()


Conectando a MongoDB en sistema1...
¡Conexión exitosa!
--- INICIANDO PROCESAMIENTO (Métrica 1.1.07 - CP vs Domicilio Empleo) ---

Procesando Documento ID: 68f81b8800535f910a29f694
--- RESULTADO DEL ANÁLISIS ---
  > [Empleo] CP: '20314', Colonia: 'GRANJA EL CARIÑAN', Municipio: 'AGUASCALIENTES'
  > Resultado de la Métrica: CUMPLE
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695
--- RESULTADO DEL ANÁLISIS ---
  > [Empleo] CP: '20190', Colonia: 'OJOCALIENTE II', Municipio: 'AGUASCALIENTES'
  > Resultado de la Métrica: NO_CUMPLE
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696
--- RESULTADO DEL ANÁLISIS ---
  > [Empleo] CP: '20000', Colonia: 'COLONIA ZONA CENTRO', Municipio: 'AGUASCALIENTES'
  > Resultado de la Métrica: CUMPLE
--------------------------

KeyboardInterrupt: 